In [ ]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.data import AUTOTUNE
from tensorflow.keras import layers, models, utils

from config import save_path, DATASETgen
from process_data import load_data

utils.set_random_seed(0)


In [ ]:
batch_size = 32
train_dataset, val_dataset, test_dataset, total_train_samples, total_val_samples, total_test_samples = load_data(DATASETgen,
                                                                                                                 target_size=(32, 32),
                                                                                                                 batch_size=batch_size,
                                                                                                                 num_classes=2)


train_dataset = train_dataset.map(lambda x, y: (x, tf.argmax(y, axis=1)),
                                  num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

val_dataset = val_dataset.map(lambda x, y: (x, tf.argmax(y, axis=1)),
                              num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

test_dataset = test_dataset.map(lambda x, y: (x, tf.argmax(y, axis=1)),
                                num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [ ]:
custom = models.Sequential([
    layers.InputLayer(input_shape=(32, 32, 3)),
    layers.Normalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
custom.summary(show_trainable=True)

In [ ]:
custom.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

In [ ]:
model = 'custom'
mode = 'custom'
epochs = 50
date_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filepath = os.path.join(save_path, f'{model}_{mode}_{epochs}ep_{date_time}.keras')

callbacks = [keras.callbacks.ModelCheckpoint(filepath=filepath, save_best_only=True),
             keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2),
             keras.callbacks.EarlyStopping(patience=5)]

steps_per_epoch = total_train_samples // batch_size
validation_steps = total_val_samples // batch_size

history = custom.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

np.save(filepath.replace('.keras', '_history.npy'), history.history)

In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
testing_steps = total_test_samples // batch_size
test_loss, test_acc = custom.evaluate(test_dataset,
                                      steps=testing_steps)
print(f"Test loss: {test_loss:.3f}")
print(f"Test accuracy: {test_acc:.3f}")